In [11]:
import Pkg
using HTTP
using JSON
using Dates
using CSV

using Parameters
using Glob
using JuliaDB
using Query

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
  [a15396b6] + OnlineStats v0.23.0
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [16]:
struct Asset
    symbol::Symbol
    Asset(s::String) = new(Symbol(s))
end

## IEX TRADING ##
abstract type API end
struct IEXTradingAPI <: API
    base::String
end
IEXTradingAPI() = IEXTradingAPI("https://ws-api.iextrading.com/1.0")

function FetchSymbols(iex::IEXTradingAPI)
    endpoint = "/ref-data/symbols"
    JSON.Parse(Request(iex,endpoint))
end

## ALPHADVANTAGE ##

struct AlphadvantageAPI <: API
    key::String
    base::String
end
AlphadvantageAPI() = AlphadvantageAPI("3J1NP4X808DF2YY8", "https://www.alphavantage.co/query?")

Format(api::AlphadvantageAPI, endpoint::String) = api.base * endpoint * "&apikey=" * api.key

function FetchHistory(api::AlphadvantageAPI, asset::Asset, method::String, outputsize::String; datatype=nothing)
    endpoint = "symbol=$(asset.symbol)&"
    endpoint *= "function=$method&"
    endpoint *= "outputsize=$outputsize"
    
    !isnothing(datatype) ? endpoint *= "&datatype=$datatype" : nothing 
    
    Request(api, endpoint)
end

FetchWeeklyHistory(api::AlphadvantageAPI, asset::Asset, outputsize::String; datatype=nothing) =
    FetchHistory(api, asset, "TIME_SERIES_WEEKLY_ADJUSTED", outputsize, datatype=datatype)

FetchDailyHistory(api::AlphadvantageAPI, asset::Asset, outputsize::String; datatype=nothing) =
    FetchHistory(api, asset, "TIME_SERIES_DAILY_ADJUSTED", outputsize, datatype=datatype)

## ASSET ##
@with_kw mutable struct AssetHistoryBuffer
    asset::Asset
    dates::Array{Date}                   = Array{Date,1}()
    open::Array{<:AbstractFloat}         = zeros()
    high::Array{<:AbstractFloat}         = zeros()
    close::Array{<:AbstractFloat}        = zeros()
    close_adjusted::Array{<:AbstractFloat} = zeros()
    low::Array{<:AbstractFloat}          = zeros()
    volume::Array{<:AbstractFloat}       = zeros()
    dividend::Array{<:AbstractFloat}     = zeros()
    split_coef::Array{<:AbstractFloat}   = zeros()
end

struct AssetHistory
    asset::Asset
    history::IndexedTable
end
AssetHistory(symbol::String) = AssetHistory(Symbol(symbol))
AssetHistory(asset::Asset) = AssetHistory(asset.symbol)

function AssetHistory(asset::Asset, df::DataFrame)
    AssetHistory(
        asset, loadtable(df, pkey = [:timestamp]))
end
 
function AssetHistory(buffer::AssetHistoryBuffer)
   history = table((timestamp=buffer.dates, open=buffer.open, high=buffer.high, close=buffer.close,
            adjusted_close=buffer.close_adjusted, low=buffer.low, volume=buffer.volume,
            dividend_amount=buffer.dividend,split_coefficient=buffer.split_coef), 
            pkey=[:timestamp]) 
    
    AssetHistory(buffer.asset.symbol, history)
end


function AddFromDate(asset::AssetHistoryBuffer, date::String, dict::Dict)
    date = int.(split(date,"-"))
    date = Date(date...)

    push!(asset.dates, date)
    push!(asset.open, _dict["1. open"])
    push!(asset.high, _dict["2. high"])
    push!(asset.low, _dict["3. low"])
    push!(asset.close, _dict["4. close"])
    push!(asset.close_adjusted, _dict["5. adjusted close"])
    push!(asset.volume, _dict["6. volume"])
    push!(asset.dividend, _dict["7. dividend amount"])
    push!(asset.split_coef, _dict["8. split coefficient"])
end

function GetFromHistory(asset::AssetHistoryBuffer, history::Dict)
    for (date,value) in history
        AddFromDate(asset,date,value)
    end
    return AssetHistory(asset)
end

function SaveStockHistory(asset::Asset; outputsize="full", API::API = AlphadvantageAPI())

    r = FetchDailyHistory(API, asset, "full", datatype="csv")

    open("resources/$(asset.symbol).csv", "w+") do io
       write(io, r) 
    end 
end

function SaveStocksHistory()
    df = CSV.File("resources/1billion_companies.tsv", delim='\t') 
    aa = AlphadvantageAPI()
    i = 0
    for row in df
        symbol = row.Symbol
        if symbol === missing
            continue
        end    

        if row.Sector == "n/a"
            continue
        end

        asset = Asset(symbol)
        SaveStockHistory(asset, outputsize="full", API=aa)

        i += 1
        if i > 100
            break
        end
        sleep(10)
    end
end


Format(api::API, endpoint::String) = api.base * endpoint
    
function Request(api::API, endpoint::String)
    url = Format(api, endpoint)
    println("Requesting $url")
    r = HTTP.request("GET", url)
    
    r.body
end



## Investment & Portfolio ##

abstract type InvestmentType end
abstract type LongInvestment <: InvestmentType end
abstract type ShortInvestment <: InvestmentType end 

struct InvestmentReturn
    value::AbstractFloat
    percentage::AbstractFloat
end

abstract type AbstractInvestment end

struct Investment{T} <: AbstractInvestment where T <: InvestmentType
    asset::Asset
    value::AbstractFloat
    dateOpen::Date
end

struct ClosedInvestment{T} <: AbstractInvestment where T <: InvestmentType
    asset::Asset
    valueOpen::AbstractFloat
    valueClose::AbstractFloat
    dateOpen::DateTime
    dateClosed::DateTime
    closed::InvestmentReturn
end


abstract type AbstractPortfolio end

struct Portfolio <: AbstractPortfolio 
    Investments::Array{<:AbstractInvestment}
end
Add!(portfolio::AbstractPortfolio, inv::AbstractInvestment) = push!(portfolio.Investments, inv)

function Long!(portfolio::AbstractPortfolio, asset::Asset, value::AbstractFloat; 
        dateOpen::Date = Dates.today(), dateClose::Union{Date,Nothing} = nothing)
    
    inv = Investment{LongInvestment}(asset, value, dateOpen, dateClose)
    add!(portfolio, inv)
end

function Short!(portfolio::AbstractPortfolio, asset::Asset, value::AbstractFloat; 
        dateOpen::Date = Dates.today(), dateClose::Union{Date,Nothing} = nothing)
    
    inv = Investment{ShortInvestment}(asset, value, dateOpen, dateClose)
    add!(portfolio, inv)
end

function Close!(pf::AbstractPortfolio, inv::Investment)
    findfirst(pf.Investments, x => x == inv)
end


Return(inv::Investment{LongInvestment}, value::AbstractFloat) = value - inv.value
Return(inv::Investment{ShortInvestment}, value::AbstractFloat) = inv.value - value

PotentialProfit(inv::AbstractInvestment) = 0.
ClosedProfit(inv::AbstractInvestment) = 0.
ClosedPercentage(inv::AbstractInvestment) = 0.

ClosedProfit(inv::ClosedInvestment) = inv.close.value
ClosedPercentage(inv::ClosedInvestment) = inv.close.percentage

function PotentialProfit(inv::Investment; currentValue::Union{AbstractFloat,Nothing} = nothing,
    dateTime::DateTime = Dates.now())
    
    if isnothing(currentValue)
        currentValue = FetchAssetValue(inv.asset, dateTime)
    end
    
    return Return(inv, currentValue) 
end

function PotentialProfit(pf::AbstractPortfolio)
    total = 0.
    for inv in pf.investestements
        total += PotentialProfit(inv)
    end
end

function ClosedProfit(pf::AbstractPortfolio)
    total = 0.
    for inv in pf.investestements
        total += ClosedProfit(inv)
    end
end

function LoadTop100History()
   assetHistories = Dict()
    for path in glob("*.csv", "resources/Top100Companies/")
        symbol = split( split(path, '/')[end], '.')[1]
        asset = Asset(String(symbol))

        @show "Loading $(symbol)"

        hist = loadtable(path, indexcols=["timestamp"])
        assetHistories[symbol] = hist
    end 
    return assetHistories
end

ClosedProfit (generic function with 3 methods)

In [7]:
# SaveStocksHistory()

Requesting https://www.alphavantage.co/query?symbol=MSFT&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=AMZN&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=AAPL&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOOGL&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOOG&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=FB&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=CSCO&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&api

Requesting https://www.alphavantage.co/query?symbol=PAYX&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=TEAM&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=MELI&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=ORLY&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=EA&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=ALXN&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOLD&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apik

"Loading $(symbol)" = "Loading AABA"
"Loading $(symbol)" = "Loading AAPL"
"Loading $(symbol)" = "Loading ADBE"
"Loading $(symbol)" = "Loading ADI"
"Loading $(symbol)" = "Loading ADP"
"Loading $(symbol)" = "Loading ADSK"
"Loading $(symbol)" = "Loading ALGN"
"Loading $(symbol)" = "Loading ALXN"
"Loading $(symbol)" = "Loading AMAT"
"Loading $(symbol)" = "Loading AMD"
"Loading $(symbol)" = "Loading AMGN"
"Loading $(symbol)" = "Loading AMOV"
"Loading $(symbol)" = "Loading AMTD"
"Loading $(symbol)" = "Loading AMZN"
"Loading $(symbol)" = "Loading ATVI"
"Loading $(symbol)" = "Loading AVGO"
"Loading $(symbol)" = "Loading BIDU"
"Loading $(symbol)" = "Loading BIIB"
"Loading $(symbol)" = "Loading BKNG"
"Loading $(symbol)" = "Loading CDNS"
"Loading $(symbol)" = "Loading CELG"
"Loading $(symbol)" = "Loading CERN"
"Loading $(symbol)" = "Loading CHTR"
"Loading $(symbol)" = "Loading CMCSA"
"Loading $(symbol)" = "Loading CME"
"Loading $(symbol)" = "Loading COST"
"Loading $(symbol)" = "Loading CSCO"
"Loa

In [28]:
JuliaDB.select(assetHistories["AMZN"],:open)

5407-element Array{Float64,1}:
   60.0 
   58.63
   56.38
   58.0 
   56.38
   56.0 
   50.0 
   52.13
   55.88
   57.5 
   59.5 
   57.88
   59.25
    ⋮   
 1866.72
 1864.0 
 1876.5 
 1901.35
 1907.84
 1933.33
 1916.1 
 1912.66
 1911.84
 1892.48
 1902.0 
 1909.1 

In [23]:
date = Date("2018-09-01")

for (symbol, hist) in assetHistories
    idx = findfirst(assetHistories, x->x.Date == date)
end



1

In [8]:
table = loadtable("resources/Top100Companies/MSFT.csv", indexcols=["timestamp"])

Table with 5407 rows, 9 columns:
Columns:
#  colname            type
─────────────────────────────
1  timestamp          Date
2  open               Float64
3  high               Float64
4  low                Float64
5  close              Float64
6  adjusted_close     Float64
7  volume             Int64
8  dividend_amount    Float64
9  split_coefficient  Float64

In [9]:
getindex(table)

MethodError: MethodError: no method matching getindex(::IndexedTable{StructArrays.StructArray{NamedTuple{(:timestamp, :open, :high, :low, :close, :adjusted_close, :volume, :dividend_amount, :split_coefficient),Tuple{Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64}},1,NamedTuple{(:timestamp, :open, :high, :low, :close, :adjusted_close, :volume, :dividend_amount, :split_coefficient),Tuple{Array{Date,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,1},Array{Float64,1},Array{Float64,1}}}}})
Closest candidates are:
  getindex(::IndexedTable, !Matched::Integer) at /home/edoardo/.julia/packages/IndexedTables/L5ua6/src/indexedtable.jl:196
  getindex(::IndexedTable, !Matched::Colon) at /home/edoardo/.julia/packages/IndexedTables/L5ua6/src/indexedtable.jl:197
  getindex(::IndexedTable, !Matched::AbstractArray{#s37,1} where #s37<:Integer) at /home/edoardo/.julia/packages/IndexedTables/L5ua6/src/indexedtable.jl:214